<a href="https://colab.research.google.com/github/AdsonRamos/if697-2020.2/blob/main/Projeto_IF697.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalando dependências

Se estiver executando este notebook no Google Colab, descomente a linha abaixo.

In [ ]:
# pip install geopandas

# Importando dependências

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import geopandas

import math

from matplotlib import pyplot as plt

# Carregando dados

In [ ]:
customers = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_customers_dataset.csv", encoding="ISO-8859-1")
geolocation = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_geolocation_dataset.csv", encoding="ISO-8859-1")
order_items = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_order_items_dataset.csv", encoding="ISO-8859-1")
order_payments = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_order_payments_dataset.csv", encoding="ISO-8859-1")
order_reviews = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_order_reviews_dataset.csv", encoding="ISO-8859-1")
orders = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_orders_dataset.csv", encoding="ISO-8859-1")
products = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_products_dataset.csv", encoding="ISO-8859-1")
sellers = pd.read_csv("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/olist_sellers_dataset.csv", encoding="ISO-8859-1")
states_maps = geopandas.read_file("https://raw.githubusercontent.com/AdsonRamos/if697-2020.2/main/datasets/br_states_shapefiles.zip")

# Descrição inicial dos datasets

In [ ]:
customers.info()

In [ ]:
geolocation.info()

In [ ]:
order_items.info()

In [ ]:
order_payments.info()

In [ ]:
order_reviews.info()

In [ ]:
orders.info()

In [ ]:
products.info()

In [ ]:
sellers.info()

Podemos ver que o dataset de geolocalizações é o maior de todos, com mais de 1 milhão de entradas. No outro extremo, temos o de vendedores, com cerca de 3 mil.

A maioria das colunas não apresenta valores nulos, mas o dataset de resenhas tem as colunas `review_comment_title` e `review_comment_message` com muitos valores nulos. Também chama a atenção os datasets de produtos e pedidos, onde algumas colunas possuem poucos valores nulos.

# Estudo sobre o dataset de clientes

In [ ]:
customers.head()

In [ ]:
customers.describe()

In [ ]:
#Estados mais frequentes nos clientes
customers['customer_state'].value_counts().plot(kind='bar', ylabel='Clientes', xlabel='Estados')

## Agrupamento por regiões

In [ ]:
sudeste = ['MG', 'RJ', 'RS', 'SP']
nordeste = ['AL', 'BA', 'CE', 'MA', 'PB', 'PI', 'PE', 'RN', 'SE']
sul = ['PR', 'RS', 'SC']
norte = ['AM', 'RR', 'AP', 'PA', 'TO', 'RO', 'AC']
centro_oeste = ['MT', 'MS', 'GO', 'DF']

In [ ]:
#Clientes distribuídos por regiões do país

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(18, 12))
fig.delaxes(axes[1,2])

northeast_client_counts = customers[customers.customer_state.isin(nordeste)]['customer_state'].value_counts().rename_axis('customer_state').reset_index(name='clients')
northeast_client_counts["customer_state"] = northeast_client_counts["customer_state"].astype('category')

north_client_counts = customers[customers.customer_state.isin(norte)]['customer_state'].value_counts().rename_axis('customer_state').reset_index(name='clients')
north_client_counts["customer_state"] = north_client_counts["customer_state"].astype('category')

south_client_counts = customers[customers.customer_state.isin(sul)]['customer_state'].value_counts().rename_axis('customer_state').reset_index(name='clients')
south_client_counts["customer_state"] = south_client_counts["customer_state"].astype('category')

southeast_client_counts = customers[customers.customer_state.isin(sudeste)]['customer_state'].value_counts().rename_axis('customer_state').reset_index(name='clients')
southeast_client_counts["customer_state"] = southeast_client_counts["customer_state"].astype('category')

midwest_client_counts = customers[customers.customer_state.isin(centro_oeste)]['customer_state'].value_counts().rename_axis('customer_state').reset_index(name='clients')
midwest_client_counts["customer_state"] = midwest_client_counts["customer_state"].astype('category')

sns.barplot(x='customer_state', y='clients', order=northeast_client_counts['customer_state'], data=northeast_client_counts, ax=axes[0][0]).set(title="Nordeste")
sns.barplot(x='customer_state', y='clients', order=north_client_counts['customer_state'], data=north_client_counts, ax=axes[0][1]).set(title="Norte")
sns.barplot(x='customer_state', y='clients', order=south_client_counts['customer_state'], data=south_client_counts, ax=axes[0][2]).set(title="Sul")
sns.barplot(x='customer_state', y='clients', order=southeast_client_counts['customer_state'], data=southeast_client_counts, ax=axes[1][0]).set(title="Sudeste")
sns.barplot(x='customer_state', y='clients', order=midwest_client_counts['customer_state'], data=midwest_client_counts, ax=axes[1][1]).set(title="Centro-Oeste")

# Estudo sobre as localizações

In [ ]:
geolocation.head()

In [ ]:
geolocation.describe()

In [ ]:
world_map = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world_map.head()

In [ ]:
points = geopandas.GeoDataFrame(geolocation, geometry=geopandas.points_from_xy(geolocation['geolocation_lng'], geolocation['geolocation_lat']))
points.head()

In [ ]:
# Localização dos pedidos em relação ao mundo
ax = world_map.plot(color='white', edgecolor='black', figsize=(25,20))
points.plot(ax=ax, color='red')
plt.show()

In [ ]:
from sklearn.neighbors import LocalOutlierFactor
clf = LocalOutlierFactor(n_neighbors=4)
predictions = clf.fit_predict(geolocation[['geolocation_lng', 'geolocation_lat']])
predictions[predictions == -1] = 0
predictions = predictions.astype('bool')

In [ ]:
outlier_points = points[predictions]
inlier_points = points[~predictions]

In [ ]:
ax = world_map.plot(color='white', edgecolor='black', figsize=(25,20))
inlier_points.plot(ax=ax, color='red')
outlier_points.plot(ax=ax, color='blue')
plt.show()

In [ ]:
# Localização dos pedidos no Brasil
brazil_map = world_map[world_map['name'] == 'Brazil']
points_in_brazil = points.within(brazil_map.iloc[0]['geometry'])
ax = brazil_map.plot(color='white', edgecolor='black', figsize=(20,20))
points[points_in_brazil].plot(ax=ax, color='red')
plt.show()

In [ ]:
# Localização dos pedidos no nordeste
fig, axes = plt.subplots(ncols=3, nrows=3, figsize=(18, 12))
for i in range(len(nordeste)):
  axes[int(i/3)][i%3].scatter(geolocation[geolocation['geolocation_state'] == nordeste[i]]['geolocation_lng'], geolocation[geolocation['geolocation_state'] == nordeste[i]]['geolocation_lat'])
  axes[int(i/3)][i%3].set_title(nordeste[i])
  axes[int(i/3)][i%3].grid(True)

In [ ]:
# Localização dos pedidos no sudeste
fig, axes = plt.subplots(ncols=4, nrows=1, figsize=(18, 4))
for i in range(len(sudeste)):
  axes[i].scatter(geolocation[geolocation['geolocation_state'] == sudeste[i]]['geolocation_lng'], geolocation[geolocation['geolocation_state'] == sudeste[i]]['geolocation_lat'])
  axes[i].set_title(sudeste[i])
  axes[i].grid(True)

In [ ]:
# Localização dos pedidos no sul
fig, axes = plt.subplots(ncols=3, nrows=1, figsize=(18, 4))
for i in range(len(sul)):
  axes[i].scatter(geolocation[geolocation['geolocation_state'] == sul[i]]['geolocation_lng'], geolocation[geolocation['geolocation_state'] == sul[i]]['geolocation_lat'])
  axes[i].set_title(sul[i])
  axes[i].grid(True)

In [ ]:
# Localização dos pedidos no norte
fig, axes = plt.subplots(ncols=3, nrows=3, figsize=(18, 12))
fig.delaxes(axes[2,1])
fig.delaxes(axes[2,2])
for i in range(len(norte)):
  axes[int(i/3)][i%3].scatter(geolocation[geolocation['geolocation_state'] == norte[i]]['geolocation_lng'], geolocation[geolocation['geolocation_state'] == norte[i]]['geolocation_lat'])
  axes[int(i/3)][i%3].set_title(norte[i])
  axes[int(i/3)][i%3].grid(True)

In [ ]:
# Localização dos pedidos no centro_oeste
fig, axes = plt.subplots(ncols=4, nrows=1, figsize=(18, 4))
for i in range(len(centro_oeste)):
  axes[i].scatter(geolocation[geolocation['geolocation_state'] == centro_oeste[i]]['geolocation_lng'], geolocation[geolocation['geolocation_state'] == centro_oeste[i]]['geolocation_lat'])
  axes[i].set_title(centro_oeste[i])
  axes[i].grid(True)

Observe que em algums gráficos, alguns pontos encontram-se bem distantes da maior parte deles. Vamos concentrar esses pontos removendo outliers usando razão de duas variáveis.

Removeremos outliers do estado de Pernambuco utilizando razão de duas variáveis

In [ ]:
geolocation_pe = geolocation[geolocation['geolocation_state'] == 'PE']
print(len(geolocation_pe))
geolocation_pe['lat_per_lng'] = geolocation_pe['geolocation_lat']/geolocation_pe['geolocation_lng']

In [ ]:
geolocation_pe['lat_per_lng'].describe()

In [ ]:
geolocation_pe['lat_per_lng'].plot.box()

In [ ]:
geolocation_pe['lat_per_lng'].hist()

In [ ]:
geolocation_pe['lat_per_lng'] = log10(geolocation_pe['lat_per_lng'])

In [ ]:
geolocation_pe['lat_per_lng'].hist()

In [ ]:
mad = abs(geolocation_pe['lat_per_lng'] - geolocation_pe['lat_per_lng'].median()).median()*(1/0.6745)

In [ ]:
(abs(geolocation_pe['lat_per_lng'] - geolocation_pe['lat_per_lng'].median())/mad).hist()

In [ ]:
geolocation_pe[abs(geolocation_pe['lat_per_lng'] - geolocation_pe['lat_per_lng'].median())/mad > 3.5]

In [ ]:
geolocation_pe = geolocation_pe[abs(geolocation_pe['lat_per_lng'] - geolocation_pe['lat_per_lng'].median())/mad < 3.5]
print(len(geolocation_pe))

# Estudo sobre os pedidos

In [ ]:
products.head()

Como existem muitas categorias, é interessante buscar aquelas cuja distribuição de alguma variável segue uma normal. Escolheremos a variável peso. E a seguir, plotaremos os respectivos boxplots e histogramas, evidenciando e o que foi suposto nos testes.

In [ ]:
# Categorias com mais de 100 registros

top_frequency_categories = products['product_category_name'].value_counts()[products['product_category_name'].value_counts() > 100].index.tolist()
tests = []

In [ ]:
for i in range(0, len(top_frequency_categories)):
  statistic, p_value = stats.shapiro(products[products['product_category_name'] == top_frequency_categories[i]]['product_weight_g'])
  x = float(statistic) 
  shapiro_test = {
      "statistic": statistic,
      "p_value": p_value,
      "category": top_frequency_categories[i]
  }
  if(not math.isnan(x)):
    tests.append(shapiro_test)

In [ ]:
tests = sorted(tests, key=lambda k: k['statistic'], reverse=True)

In [ ]:
tests[:5]

In [ ]:
# Histogramas dos 5 principais testes
fig, axes = plt.subplots(ncols=5, nrows=1, figsize=(18, 4))
for i in range(5):
  axes[i].hist(products[products['product_category_name'] == tests[:5][i]['category']]['product_weight_g'])
  axes[i].set_title(tests[:5][i]['category'])
  axes[i].grid(True)

In [ ]:
# Boxplot dos 5 principais testes
fig, axes = plt.subplots(ncols=5, nrows=1, figsize=(18, 4))
for i in range(5):
  axes[i].boxplot(products[products['product_category_name'] == tests[:5][i]['category']]['product_weight_g'])
  axes[i].set_title(tests[:5][i]['category'])
  axes[i].grid(True)

Para cada uma das categorias descritas acima, vamos remover alguns outliers utilizando técnicas de remoção univariadas

# Removendo outlier utilizando z-score robusto

Para remover outliers, vamo escolher um subconjunto cuja distribuição dos dados esteja bastante esparsa. Por exemplo, escolheremos o 25º resultado dos testes obtidos na seção anterior.

In [ ]:
from numpy import log10
pesos = products[products['product_category_name'] == tests[25]['category']]['product_weight_g']
print(len(pesos))

In [ ]:
pesos_log = log10(pesos)
pesos_log.hist()

In [ ]:
from numpy import abs
mad = abs(pesos_log - pesos_log.median()).median()*(1/0.6745)
print(mad)

In [ ]:
pesos_log.mad()

In [ ]:
(abs(pesos_log - pesos_log.median())/mad).hist()

In [ ]:
pesos_depois = pesos[abs(pesos_log - pesos_log.median())/mad < 3.5]
len(pesos_depois)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(18, 12))
axes[0][0].hist(pesos)
axes[0][0].set_title('Antes da remoção')
axes[0][0].grid(True)
axes[0][1].hist(pesos_depois)
axes[0][1].set_title('Depois da remoção')
axes[0][1].grid(True)
axes[1][0].boxplot(pesos)
axes[1][0].set_title('Antes da remoção')
axes[1][0].grid(True)
axes[1][1].boxplot(pesos_depois)
axes[1][1].set_title('Depois da remoção')
axes[1][1].grid(True)

# Removendo outliers utilizando método de tukey

Vamos escolher a mesma distribuição do caso anterior, a fim de comparar os resultados no final

In [ ]:
pesos = products[products['product_category_name'] == tests[25]['category']]['product_weight_g']
print(len(pesos))

In [ ]:
pesos.plot.box()

In [ ]:
pesos.hist()

In [ ]:
pesos_log = log10(pesos)
pesos_log.hist()

In [ ]:
q1 = pesos_log.quantile(q=0.25)
q3 = pesos_log.quantile(q=0.75)
iqr = q3 - q1
print(iqr)

In [ ]:
pesos[pesos_log < (q1-1.5*iqr)]

In [ ]:
pesos[pesos_log > (q3+1.5*iqr)]

In [ ]:
pesos_depois = pesos[abs(pesos_log - pesos_log.median())/mad < 3.5]
len(pesos_depois)

In [ ]:
fig, axes = plt.subplots(ncols=2, nrows=2, figsize=(18, 12))
axes[0][0].hist(pesos)
axes[0][0].set_title('Antes da remoção')
axes[0][0].grid(True)
axes[0][1].hist(pesos_depois)
axes[0][1].set_title('Depois da remoção')
axes[0][1].grid(True)
axes[1][0].boxplot(pesos)
axes[1][0].set_title('Antes da remoção')
axes[1][0].grid(True)
axes[1][1].boxplot(pesos_depois)
axes[1][1].set_title('Depois da remoção')
axes[1][1].grid(True)

Observe que para essa mesma distribuição de dados, o z-score robusto e o método de Tukey removem a mesma quantidade de dados.